# Bathroom Ventilation System Analysis

This notebook analyzes the ventilation system for a multi-floor facility's bathrooms. It calculates required airflow rates, duct velocities, and recommends appropriate centrifugal fans.

In [ ]:
# Import the ventilation calculator module
from ventilation_calculator import *

## Analyze the Ventilation System

We'll analyze the system under the worst-case climate conditions (hot and humid).

In [ ]:
# Analyze for worst climate case
climate_condition = 'extreme_hot_humid'
results, fan_requirements = analyze_ventilation_system(climate_condition)

## Generate Improvement Suggestions

In [ ]:
# Generate suggestions for improvements
suggestions = suggest_improvements(results)

# Display suggestions
for bathroom_name, bathroom_suggestions in suggestions.items():
    if bathroom_suggestions:
        print(f"\n{bathroom_name}:")
        for suggestion in bathroom_suggestions:
            print(f"  - {suggestion}")

## Recommend Fans

In [ ]:
# Recommend fans
fan_recommendations = recommend_fans(fan_requirements)

# Display fan recommendations
for system, recommendation in fan_recommendations.items():
    print(f"\n{system.upper()}:")
    if isinstance(recommendation, dict):
        print(f"  Recommended model: {recommendation['model']}")
        print(f"  Maximum flow: {recommendation['max_flow_cfm']:.1f} CFM")
        print(f"  Maximum pressure: {recommendation['max_pressure_inwg']:.4f} inWG")
        print(f"  Power consumption: {recommendation['power_watts']} W")
        print(f"  Flow margin: {recommendation['flow_margin']:.1f}%")
        print(f"  Pressure margin: {recommendation['pressure_margin']:.1f}%")
    else:
        print(f"  {recommendation}")

## Generate Full Report

In [ ]:
# Generate report
generate_report(results, fan_requirements, suggestions, fan_recommendations, climate_condition)

## Visualize the System

In [ ]:
# Plot system diagram
plot_system_diagram(results, fan_recommendations)

# Display the diagram
from IPython.display import Image
Image(filename='ventilation_system_diagram.png')

## Save Results to CSV

In [ ]:
# Save results to CSV
results_df = pd.DataFrame()
for bathroom_name, data in results.items():
    row = {
        'Bathroom': bathroom_name,
        'System': data['system']
    }
    
    if bathroom_name != 'Ducto_Principal_Piso2':
        row.update({
            'Volume (m³)': data['volume_m3'],
            'Air Changes per Hour': data['air_changes_per_hour'],
            'Grille Velocity (m/s)': data['grille_velocity_m_s']
        })
    
    row.update({
        'Flow Rate (m³/h)': data['flow_rate_m3h'],
        'Flow Rate (CFM)': data['flow_rate_cfm'],
        'Total Pressure Loss (Pa)': data['total_pressure_loss_pa']
    })
    
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

results_df.to_csv('ventilation_results.csv', index=False)
results_df

In [ ]:
# Save fan recommendations to CSV
fan_df = pd.DataFrame()
for system, recommendation in fan_recommendations.items():
    if isinstance(recommendation, dict):
        row = {
            'System': system,
            'Model': recommendation['model'],
            'Max Flow (CFM)': recommendation['max_flow_cfm'],
            'Max Pressure (inWG)': recommendation['max_pressure_inwg'],
            'Power (W)': recommendation['power_watts'],
            'Flow Margin (%)': recommendation['flow_margin'],
            'Pressure Margin (%)': recommendation['pressure_margin']
        }
    else:
        row = {
            'System': system,
            'Model': 'Not found',
            'Max Flow (CFM)': None,
            'Max Pressure (inWG)': None,
            'Power (W)': None,
            'Flow Margin (%)': None,
            'Pressure Margin (%)': None
        }
    
    fan_df = pd.concat([fan_df, pd.DataFrame([row])], ignore_index=True)

fan_df.to_csv('fan_recommendations.csv', index=False)
fan_df

## Interactive Adjustments

Use this section to interactively adjust parameters and see the impact on the ventilation system.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def update_bathroom(bathroom_name, volume, ach):
    # Update the bathroom parameters
    if bathroom_name in bathrooms and bathroom_name != 'Ducto_Principal_Piso2':
        bathrooms[bathroom_name]['volume_m3'] = volume
        
        # Recalculate and display results
        clear_output(wait=True)
        results, fan_requirements = analyze_ventilation_system(climate_condition)
        suggestions = suggest_improvements(results)
        fan_recommendations = recommend_fans(fan_requirements)
        
        # Display updated results for this bathroom
        print(f"Updated {bathroom_name}:")
        print(f"  Volume: {results[bathroom_name]['volume_m3']:.1f} m³")
        print(f"  Air changes per hour: {results[bathroom_name]['air_changes_per_hour']:.1f}")
        print(f"  Flow rate: {results[bathroom_name]['flow_rate_m3h']:.1f} m³/h ({results[bathroom_name]['flow_rate_cfm']:.1f} CFM)")
        print(f"  Grille velocity: {results[bathroom_name]['grille_velocity_m_s']:.2f} m/s")
        
        # Display suggestions if any
        if bathroom_name in suggestions and suggestions[bathroom_name]:
            print("\nSuggestions:")
            for suggestion in suggestions[bathroom_name]:
                print(f"  - {suggestion}")
        else:
            print("\nNo suggestions - parameters are within recommended ranges.")

# Create dropdown for bathroom selection
bathroom_dropdown = widgets.Dropdown(
    options=[name for name in bathrooms if name != 'Ducto_Principal_Piso2'],
    description='Bathroom:',
    disabled=False,
)

# Create sliders for volume and ACH
volume_slider = widgets.FloatSlider(
    value=10.0,
    min=5.0,
    max=30.0,
    step=0.5,
    description='Volume (m³):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

ach_slider = widgets.FloatSlider(
    value=10.0,
    min=6.0,
    max=20.0,
    step=0.5,
    description='ACH:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Update function for when bathroom is selected
def on_bathroom_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        bathroom_name = change['new']
        if bathroom_name in bathrooms:
            # Update sliders to current values
            volume_slider.value = bathrooms[bathroom_name]['volume_m3']

bathroom_dropdown.observe(on_bathroom_change)

# Create update button
update_button = widgets.Button(
    description='Update Parameters',
    disabled=False,
    button_style='', 
    tooltip='Click to update parameters',
    icon='check'
)

# Button click handler
def on_button_clicked(b):
    update_bathroom(bathroom_dropdown.value, volume_slider.value, ach_slider.value)

update_button.on_click(on_button_clicked)

# Display widgets
display(bathroom_dropdown, volume_slider, ach_slider, update_button)